<a href="https://colab.research.google.com/github/Sandeepkumaramgothu/ASE_practice/blob/main/Spect_Create.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install --upgrade keras
!pip install --upgrade keras tensorflow
import os
import math
import numpy as np
import librosa
import json
import pandas as pd
from termcolor import colored
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
!pip install tensorboard
!pip install visualkeras
import visualkeras
from keras.utils import plot_model
from datetime import datetime
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn import svm
from sklearn.svm import SVC
import pickle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from IPython.display import Audio
from scipy.io import wavfile
import scipy
import soundfile as sf
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report
import random

from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.4/997.4 kB 22.1 MB/s eta 0:00:00
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import os
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt

def create_mel_spectrogram(audio_file, output_dir, sr=16000, n_fft=2048, hop_length=512, n_mels=128):
    y, sr = librosa.load(audio_file, sr=sr)  # Load the audio file with the given sample rate
    mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)  # Create mel spectrogram
    mel_spec_db = librosa.amplitude_to_db(mel_spec, ref=np.max)  # Convert the amplitude to decibels

    # Plot the spectrogram
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(mel_spec_db, sr=sr, hop_length=hop_length, x_axis='time', y_axis='mel')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Mel Spectrogram')
    plt.tight_layout()

    # Save the plot to the specified directory
    filename = os.path.basename(audio_file).split('.')[0] + '_mel.png'
    save_path = os.path.join(output_dir, filename)
    plt.savefig(save_path)
    plt.close()
    print(f'Mel spectrogram saved as {filename}')

if __name__ == '__main__':
    audio_dir = '/content/drive/MyDrive/DataSet/Sound/'
    output_dir = '/content/drive/MyDrive/DataSet/Mel_Images/'

    # Ensure the base output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Traverse the audio directory
    for root, dirs, files in os.walk(audio_dir):
        for file in files:
            if file.endswith('.wav') or file.endswith('.mp3'):
                audio_file = os.path.join(root, file)
                # Define the output subdirectory based on the category
                category = os.path.basename(os.path.dirname(audio_file))
                category_output_dir = os.path.join(output_dir, category)
                os.makedirs(category_output_dir, exist_ok=True)  # Ensure the output directory exists
                create_mel_spectrogram(audio_file, category_output_dir)


Mel spectrogram saved as firework-accident-76685_mel.png
Mel spectrogram saved as error-170796_mel.png
Mel spectrogram saved as electronic-crash-6-47446_mel.png
Mel spectrogram saved as electronic-crash-8-100412_mel.png
Mel spectrogram saved as electronic-crash-3-47448_mel.png
Mel spectrogram saved as electronic-crash-10-100410_mel.png
Mel spectrogram saved as dishes-fall-and-crash-117075_mel.png
Mel spectrogram saved as cristal-43446_mel.png
Mel spectrogram saved as electronic-crash-10-100410 (1)_mel.png
Mel spectrogram saved as crash-fx-231813_mel.png
Mel spectrogram saved as crash-verb-89638 (1)_mel.png
Mel spectrogram saved as crash-glass-sound-effect-24-11503_mel.png
Mel spectrogram saved as crash-verb-89638_mel.png
Mel spectrogram saved as crash-fluctor-106391_mel.png
Mel spectrogram saved as crash-down-some-stairs-106946_mel.png
Mel spectrogram saved as tr808-crash-cymbal-241377_mel.png
Mel spectrogram saved as tr707-crash-cymbal-241376_mel.png
Mel spectrogram saved as train-1-9

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = '/content/drive/MyDrive/Colab Notebooks/TAMUCC REU 2024/DS1/Al-Emadi_Standardized_Mel_Spectrograms/Train/'
test_dir = '/content/drive/MyDrive/Colab Notebooks/TAMUCC REU 2024/DS1/Al-Emadi_Standardized_Mel_Spectrograms/Test/'

num_classes = 8
height = 128
n_fft = 2048
hop_length = 512
sr = 16000
T = 1

hop_size_sec = hop_length / sr
audio_length_samples = T * sr

width = 1 + int((audio_length_samples - n_fft) / hop_length)

channels = 3

train_datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator()

batch_size = 32

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical')

model = tf.keras.Sequential()

# Convolutional layers
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(height, width, channels)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))

# Flatten layer to feed into fully connected layers
model.add(tf.keras.layers.Flatten())

# Dense layers
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

epochs = 200
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    train_generator.reset()

    model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=1,
        verbose=1)

loss, accuracy = model.evaluate(test_generator)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

model.save('el-amadi_2000_cnn_mcc_standardized.keras')

In [ ]:
#New Testing

In [4]:
import os
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import shutil

def create_mel_spectrogram(audio_file, output_dir):
    sr = 22050
    n_fft = 2048
    hop_length = 512
    n_mels = 128

    y, sr = librosa.load(audio_file, sr=sr)
    mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
    mel_spec_db = librosa.amplitude_to_db(mel_spec, ref=np.max)

    plt.figure(figsize=(10, 4))
    librosa.display.specshow(mel_spec_db, sr=sr, hop_length=hop_length, x_axis='time', y_axis='mel')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Mel Spectrogram')
    plt.tight_layout()

    filename = os.path.basename(audio_file).split('.')[0] + '_mel.png'
    save_path = os.path.join(output_dir, filename)
    plt.savefig(save_path)
    plt.close()

    print(f'Mel spectrogram saved as {filename}')

def split_data(audio_dir, output_dir, train_ratio=0.8):
    categories = [d for d in os.listdir(audio_dir) if os.path.isdir(os.path.join(audio_dir, d))]
    print(f"Categories: {categories}")

    for category in categories:
        category_dir = os.path.join(audio_dir, category)
        files = [f for f in os.listdir(category_dir) if f.endswith('.wav') or f.endswith('.mp3')]

        np.random.shuffle(files)
        split_idx = int(len(files) * train_ratio)

        train_files = files[:split_idx]
        test_files = files[split_idx:]

        train_output = os.path.join(output_dir, 'Train', category)
        test_output = os.path.join(output_dir, 'Test', category)

        os.makedirs(train_output, exist_ok=True)
        os.makedirs(test_output, exist_ok=True)

        for f in train_files:
            create_mel_spectrogram(os.path.join(category_dir, f), train_output)

        for f in test_files:
            create_mel_spectrogram(os.path.join(category_dir, f), test_output)

if __name__ == '__main__':
    audio_dir = '/content/drive/MyDrive/DataSet/Sound/'
    output_dir = '/content/drive/MyDrive/DataSet/Mel_Images/'

    split_data(audio_dir, output_dir)


Categories: ['Medical_Emergencies', 'House_breaking', 'Emergency_alarm', 'Fights', 'Natural_disaster', 'Gun_shots', 'Explosion', 'Accident']
Mel spectrogram saved as heavy-breath-male-63980_mel.png
Mel spectrogram saved as breathing-6091_mel.png
Mel spectrogram saved as heavy-breathing-breathing-illness-sound-of-breathing-14248_mel.png
Mel spectrogram saved as heavy_breathing-34377_mel.png
Mel spectrogram saved as breathing-fast-247449_mel.png
Mel spectrogram saved as heavy-breathing-14431_mel.png
Mel spectrogram saved as gas-mask-breath-69348_mel.png
Mel spectrogram saved as choking-244324_mel.png
Mel spectrogram saved as male-heavy-breathing-67325_mel.png
Mel spectrogram saved as 044538_female-heavy-breathing-62329_mel.png
Mel spectrogram saved as real-choking-cough2-104438_mel.png
Mel spectrogram saved as real-choking-cough-85612_mel.png
Mel spectrogram saved as panting-7108_mel.png
Mel spectrogram saved as person-knocked-down-14798_mel.png
Mel spectrogram saved as heavy-breathing-s

In [6]:
import os
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf


train_dir = '/content/drive/MyDrive/DataSet/Mel_Images/Train/'
test_dir = '/content/drive/MyDrive/DataSet/Mel_Images/Test/'
num_classes = 8
height = 128
width = 173  # Calculated width based on the audio settings
channels = 1

train_datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
test_datagen = ImageDataGenerator()

batch_size = 32

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='grayscale')  # Adjusted to match the mel spectrograms

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='grayscale')

# Define and compile the CNN model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(height, width, channels)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
epochs = 200
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=1,
        verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Save the model
model.save('el-amadi_2000_cnn_mcc_standardized.keras')

Found 275 images belonging to 8 classes.
Found 72 images belonging to 8 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


9/9 ━━━━━━━━━━━━━━━━━━━━ 12s 710ms/step - accuracy: 0.1360 - loss: 69.5754
Epoch 2/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 391ms/step - accuracy: 0.1551 - loss: 2.1924
Epoch 3/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 370ms/step - accuracy: 0.2425 - loss: 1.9901
Epoch 4/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 365ms/step - accuracy: 0.3055 - loss: 1.8701
Epoch 5/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 365ms/step - accuracy: 0.3184 - loss: 1.8868
Epoch 6/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 458ms/step - accuracy: 0.2949 - loss: 1.9122
Epoch 7/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 375ms/step - accuracy: 0.2502 - loss: 1.7929
Epoch 8/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 398ms/step - accuracy: 0.2960 - loss: 1.8453
Epoch 9/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 398ms/step - accuracy: 0.3501 - loss: 1.6680
Epoch 10/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 372ms/step - accuracy: 0.3675 - loss: 1.7938
Epoch 11/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 522ms/step - accuracy: 0.3614 - loss: 1.7469
Epoch 12/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 372ms/step - accuracy: 0.4181 - 

In [10]:
# Continue from the previous script where the model is defined, compiled, trained, and saved.

# Import necessary libraries for evaluation
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Load the previously saved model
model = tf.keras.models.load_model('/content/el-amadi_2000_cnn_mcc_standardized.keras')

# Setup for data generation for testing
test_dir = '/content/drive/MyDrive/DataSet/Mel_Images/Test/'
test_datagen = ImageDataGenerator()
width = 173 # This needs to be set. In ipython-input-6-69b7f413ad7a the variable is set; however, since the evaluate script is run in a separate cell/script, the variable was never passed or assigned leading to the issue.

# Create the test generator
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(height, width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    color_mode='grayscale' # Add color_mode='grayscale'
)

# Evaluate the model on the test data
loss, accuracy = model.evaluate(test_generator)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Predict classes with the model on the test data
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Retrieve the true class labels from the test generator
true_classes = test_generator.classes

# Compute and print the confusion matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("\nConfusion Matrix:")
print(conf_matrix)

# Get the labels from the test generator for creating a classification report
class_labels = list(test_generator.class_indices.keys())

# Calculate and print accuracy for each class
class_accuracy = {}
for i, class_label in enumerate(class_labels):
    correct_indices = np.where(true_classes == i)[0]
    correct_predictions = np.sum(predicted_classes[correct_indices] == i)
    total_samples = len(correct_indices)
    class_accuracy[class_label] = correct_predictions / total_samples if total_samples > 0 else 0.0

print("\nAccuracy for each class:")
for class_label in class_labels:
    print(f"{class_label}: {class_accuracy[class_label]:.4f}")

# Generate and print the classification report
print("\nClassification Report:")
print(classification_report(true_classes, predicted_classes, target_names=class_labels))

Found 72 images belonging to 8 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 217ms/step - accuracy: 0.4336 - loss: 5.6051
Test Loss: 6.764711856842041
Test Accuracy: 0.375
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 300ms/step

Confusion Matrix:
[[9 3 0 0 0 1 1 0]
 [4 7 0 0 1 1 0 0]
 [1 0 4 1 1 0 0 0]
 [2 0 0 1 2 0 0 0]
 [7 0 1 0 1 2 0 2]
 [3 2 0 1 1 3 0 1]
 [1 1 0 0 0 1 1 0]
 [2 2 0 0 0 0 0 1]]

Accuracy for each class:
Accident: 0.6429
Emergency_alarm: 0.5385
Explosion: 0.5714
Fights: 0.2000
Gun_shots: 0.0769
House_breaking: 0.2727
Medical_Emergencies: 0.2500
Natural_disaster: 0.2000

Classification Report:
                     precision    recall  f1-score   support

           Accident       0.31      0.64      0.42        14
    Emergency_alarm       0.47      0.54      0.50        13
          Explosion       0.80      0.57      0.67         7
             Fights       0.33      0.20      0.25         5
          Gun_shots       0.17      0.08      0.11        13
     House_breaking       0.38      0.27      0.32        11
Medical_Emergencies 